# Сравниваем модели и сохраняем в `src/models/pretrained`

- Импорты
- Константы
- Считывание датасетов

In [1]:
import os
import time
import torch
import warnings
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
for warn in [UserWarning, FutureWarning]: warnings.filterwarnings("ignore", category = warn)

from src.data_utils.config import DatasetConfig
from src.data_utils.dataset_params import DatasetName
from src.data_utils.dataset_generator import DatasetGenerator
from src.models.models import TransformerClassifier, CustomMambaClassifier, LSTMClassifier

MAX_SEQ_LEN = 300
EMBEDDING_DIM = 128
BATCH_SIZE = 32
LEARNING_RATE = 1e-4
NUM_EPOCHS = 5  # для быстрого сравнения моделей
NUM_CLASSES = 2

SAVE_DIR = "../pretrained_comparison"
os.makedirs(SAVE_DIR, exist_ok=True)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

config = DatasetConfig(
    load_from_disk=True,
    path_to_data="../datasets"
)

generator = DatasetGenerator(DatasetName.IMDB, config=config)
(X_train, y_train), (X_val, y_val), (X_test, y_test) = generator.generate_dataset()
VOCAB_SIZE = len(generator.vocab)

Вспомогательные функции для трейна/валидации/теста

In [2]:

def train_and_evaluate(model, train_loader, val_loader, optimizer, criterion, num_epochs, device, model_name, save_path):
    best_val_f1 = 0.0
    history = {'train_loss': [], 'val_loss': [], 'val_accuracy': [], 'val_f1': []}
    
    print(f"--- Начало обучения модели: {model_name} на устройстве {device} ---")

    for epoch in range(num_epochs):
        model.train()
        start_time = time.time()
        total_train_loss = 0

        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
        
        avg_train_loss = total_train_loss / len(train_loader)
        history['train_loss'].append(avg_train_loss)

        model.eval()
        total_val_loss = 0
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                total_val_loss += loss.item()
                
                _, predicted = torch.max(outputs.data, 1)
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(batch_y.cpu().numpy())
        
        avg_val_loss = total_val_loss / len(val_loader)
        
        accuracy = accuracy_score(all_labels, all_preds)
        precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary')
        
        history['val_loss'].append(avg_val_loss)
        history['val_accuracy'].append(accuracy)
        history['val_f1'].append(f1)

        epoch_time = time.time() - start_time
        print(f"Эпоха {epoch+1}/{num_epochs} | Время: {epoch_time:.2f}с | Train Loss: {avg_train_loss:.4f} | "
              f"Val Loss: {avg_val_loss:.4f} | Val Acc: {accuracy:.4f} | Val F1: {f1:.4f}")

        if f1 > best_val_f1:
            best_val_f1 = f1
            torch.save(model.state_dict(), save_path)
            print(f"  -> Модель сохранена, новый лучший Val F1: {best_val_f1:.4f}")
            
    print(f"--- Обучение модели {model_name} завершено ---")
    return history

def evaluate_on_test(model, test_loader, device, criterion):
    model.eval()
    total_test_loss = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            total_test_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(batch_y.cpu().numpy())
            
    avg_test_loss = total_test_loss / len(test_loader)
        
    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary')
    
    return {'loss': avg_test_loss, 'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1_score': f1}

Создание даталоадера

In [3]:
def create_dataloader(X, y, batch_size, shuffle=True):
    X_tensor = torch.as_tensor(X, dtype=torch.long)
    y_tensor = torch.as_tensor(y, dtype=torch.long)
    dataset = TensorDataset(X_tensor, y_tensor)
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

train_loader = create_dataloader(X_train, y_train, BATCH_SIZE)
val_loader = create_dataloader(X_val, y_val, BATCH_SIZE, shuffle=False)
test_loader = create_dataloader(X_test, y_test, BATCH_SIZE, shuffle=False)

Сравнения моделей

Смотрим первые 5 эпох чтобы выбрать лучшую модель, с которой будем играться дальше

In [4]:
model_configs = {
    "CustomMamba": {
        "class": CustomMambaClassifier,
        "params": {'vocab_size': VOCAB_SIZE, 'd_model': EMBEDDING_DIM, 'd_state': 8, 
                   'd_conv': 4, 'num_layers': 2, 'num_classes': NUM_CLASSES},
    },

    "Lib_LSTM": {
        "class": LSTMClassifier,
        "params": {'vocab_size': VOCAB_SIZE, 'embed_dim': EMBEDDING_DIM, 'hidden_dim': 128, 
                   'num_layers': 2, 'num_classes': NUM_CLASSES, 'dropout': 0.5},
    },
    "Lib_Transformer": {
        "class": TransformerClassifier,
        "params": {'vocab_size': VOCAB_SIZE, 'embed_dim': EMBEDDING_DIM, 'num_heads': 4, 
                   'num_layers': 2, 'num_classes': NUM_CLASSES, 'max_seq_len': MAX_SEQ_LEN},
    },
}

results = {}
for model_name, config in model_configs.items():

    model_path = os.path.join(SAVE_DIR, f"best_model_{model_name.lower()}.pth")
    
    model = config['class'](**config['params']).to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    criterion = nn.CrossEntropyLoss()
    
    train_and_evaluate(
        model=model, train_loader=train_loader, val_loader=val_loader,
        optimizer=optimizer, criterion=criterion, num_epochs=NUM_EPOCHS,
        device=DEVICE, model_name=model_name, save_path=model_path
    )
    
    print(f"--- Оценка лучшей модели {model_name} на тестовых данных ---")
    if os.path.exists(model_path):
        best_model = config['class'](**config['params']).to(DEVICE)
        best_model.load_state_dict(torch.load(model_path))
        test_metrics = evaluate_on_test(best_model, test_loader, DEVICE, criterion)
        results[model_name] = test_metrics
        print(f"Результаты для {model_name}: {test_metrics}")
    else:
        print(f"Файл лучшей модели для {model_name} не найден. Пропускаем оценку.")

    print("-" * 60)
    
if results:
    results_df = pd.DataFrame(results).T
    print("\n\n--- Итоговая таблица сравнения моделей на тестовых данных ---")
    print(results_df.to_string())
else:
    print("Не удалось получить результаты ни для одной модели.")


--- Начало обучения модели: CustomMamba на устройстве cuda ---
Эпоха 1/5 | Время: 337.85с | Train Loss: 0.6768 | Val Loss: 0.6168 | Val Acc: 0.6592 | Val F1: 0.5937
  -> Модель сохранена, новый лучший Val F1: 0.5937
Эпоха 2/5 | Время: 345.54с | Train Loss: 0.5266 | Val Loss: 0.4964 | Val Acc: 0.7580 | Val F1: 0.7552
  -> Модель сохранена, новый лучший Val F1: 0.7552
Эпоха 3/5 | Время: 343.23с | Train Loss: 0.4329 | Val Loss: 0.4586 | Val Acc: 0.7812 | Val F1: 0.7830
  -> Модель сохранена, новый лучший Val F1: 0.7830
Эпоха 4/5 | Время: 342.62с | Train Loss: 0.3730 | Val Loss: 0.4596 | Val Acc: 0.7928 | Val F1: 0.8056
  -> Модель сохранена, новый лучший Val F1: 0.8056
Эпоха 5/5 | Время: 340.21с | Train Loss: 0.3127 | Val Loss: 0.4469 | Val Acc: 0.7996 | Val F1: 0.8124
  -> Модель сохранена, новый лучший Val F1: 0.8124
--- Обучение модели CustomMamba завершено ---
--- Оценка лучшей модели CustomMamba на тестовых данных ---
Результаты для CustomMamba: {'loss': 0.44949763529239944, 'accurac

По результатам видно, что LSTM и Transformer обучаются быстро, но Mamba обучается хорошо. Дальнейшие шаги следующие 
 - Пробуем сравнить Transformer и Mamba более детально, играем с гиперпараметрами
 - LSTM проигрывает Transformer и по времени, и по качеству, поэтому в следующий этап сравнения не пойдет
 
Цель следующего иследования: найти идеальный баланс между временем и качеством. Поставим больше эпох, меньший lr для обоих моделей, увеличим датасет (в текущем сетапе было 10'000 сэмплов на трейн и по 5'000 на валидацию/тест)